# Generate summed flag count tables

This notebook creates QAQC flag counts csv files per network from the corresponding eraqc_counts_timestep files that were generated as a part of the final processing step for stations within the Historical Data Pipeline. These tables are used to then generate statistics for the QAQC success report.

This is carried out in two steps:

1. Generate the per-network QAQC flag count tables, at native and hourly timesteps

2. Generates one flag count table that sums all per-network tables, at native and hourly timesteps


Using the following functions:


- _pairwise_sum(): helper function that merges two input flag tables, used by network_sum_flag_counts() and total_sum_flag_counts().

- network_sum_flag_counts(): sums all station flag count tables for a given network, creating one flag count table for that network

- generate_station_tables(): runs network_sum_flag_counts() for every network

- total_sum_flag_counts(): sums all network flag count tables, creating one final flag count table 

## Step 0: Environment set-up

In [37]:
import time
import boto3
import numpy as np
import pandas as pd
import xarray as xr

from qaqc_generate_flag_sum import *

## Step 1: Check

First, loop through every network, combining each of their station flag count tables into one table. The result is one flag count table at each timestep - native and hourly - for every network.

This will take around 1 hour to run for both timesteps. 

In [39]:
def _pairwise_sum(flag_df_1, flag_df_2) -> pd.DataFrame:
    """
    Sums two input flag count dataframes. This is a helper function for sum_flag_counts().

    Parameters
    ----------
    flag_df_1: pd.DataFrame
        dataframe of previously summed station flag counts
    flag_df_2: pd.DataFrame
        flag counts dataframes for next station

    Returns
    -------
    summed_df: pd.DataFrame

    """
    if len(flag_df_1) == 0:
        return flag_df_2
    else:
        total_df = pd.concat([flag_df_1, flag_df_2])

        summed_df = total_df.groupby("eraqc_flag_values", as_index=False).sum(
            numeric_only=True
        )
        return summed_df

In [40]:
def _total_format_table(summed_counts: pd.DataFrame) -> pd.DataFrame:
    """
    A helper function that formats the final, 'total' counts table

    Parameters
    ----------
    summed_counts: pd.DataFrame
        dataframe of summed station flag counts

    Returns
    -------
    final_format: pd.DataFrame

    """

    # convert flag value strings to integers

    summed_counts = summed_counts.applymap(
        lambda x: int(x) if not isinstance(x, str) else x
    )

    ## Format final dataframe

    # order by flag value, in descending numerical order
    final_format = (
        summed_counts.groupby(
            summed_counts.eraqc_flag_values.apply(type) != str, group_keys=True
        )
        .apply(lambda g: g.sort_values("eraqc_flag_values"))
        .reset_index(drop=True)
    )

    # move string flag value entries to the bottom
    final_format = final_format.loc[
        pd.to_numeric(final_format["eraqc_flag_values"], errors="coerce")
        .sort_values()
        .index
    ]

    return final_format

In [ ]:
test_network = pd.read_csv(
    "s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/VCAPCD_flag_counts_hourly_timestep.csv"
)
test_network.columns.isin(['svp','pvp'])

# SCAN and SNOTEL contains pvp and svp

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [8]:
test_network.columns

Index(['eraqc_flag_values', 'accum_pr_24h', 'accum_pr_localmid', 'elevation',
       'hurs', 'pr_24h', 'pr_5min', 'pr', 'pr_localmid', 'ps_altimeter',
       'ps_derived', 'ps', 'rsds', 'sfcWind_dir', 'sfcWind', 'tas',
       'tdps_derived', 'accum_pr', 'pr_1h', 'accum_pr_1h', 'QAQC_function',
       'Flag_meaning'],
      dtype='object')

In [4]:
test_total = pd.read_csv("s3://wecc-historical-wx/4_merge_wx/total_flag_counts_hourly_timestep.csv")

In [9]:
test_total.columns

Index(['eraqc_flag_values', 'elevation', 'pr', 'ps_altimeter', 'psl',
       'sfcWind_dir', 'sfcWind', 'tas', 'tdps', 'hurs_derived', 'ps',
       'accum_pr', 'rsds', 'hurs', 'ps_derived', 'tdps_derived', 'pr_5min',
       'pvp_derived', 'pr_1h', 'accum_pr_1h', 'accum_pr_5min', 'accum_pr_24h',
       'accum_pr_localmid', 'pr_24h', 'pr_localmid', 'pvp', 'svp'],
      dtype='object')

In [43]:
df_1 = pd.read_csv("s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/SNOTEL_flag_counts_hourly_timestep.csv")
df_2 = pd.read_csv(
    "s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/ASOSAWOS_flag_counts_hourly_timestep.csv"
)

In [145]:
flag_table = pd.read_csv("era_qaqc_flag_meanings.csv")

In [212]:
def _format_table(
    summed_counts: pd.DataFrame, flag_table: pd.DataFrame, level: str
) -> pd.DataFrame:
    """
    A helper function that formats the network-level counts tables

    Parameters
    ----------
    summed_counts: pd.DataFrame
        dataframe of summed station flag counts
    flag_table: pd.DataFrame
        flag counts dataframes for next station
    level: str
        either 'total' or 'network'

    Returns
    -------
    final_format: pd.DataFrame

    """
    if level not in ("network", "total"):
        print("invalid level: ", level)
        return None

    ## Format flag meanings df
    flag_table = flag_table.rename(columns={"Flag_value": "eraqc_flag_values"})

    ## Format summed counts df

    if level == 'network':
        # remove the ".0" from the flag values
        summed_counts["eraqc_flag_values"] = summed_counts["eraqc_flag_values"].str.replace(
            ".0", "", regex=True
        )

    # convert flag value strings to integers
    summed_counts["eraqc_flag_values"] = summed_counts["eraqc_flag_values"].apply(
        lambda x: int(x) if x not in ["no_flag", "total_obs_count"] else x
    )

    ## Merge the the counts and flag meanings dataframes
    merged_dfs = summed_counts.merge(
        flag_table, on="eraqc_flag_values", how="outer"
    ).fillna(0)

    ## Format final dataframe

    # order by flag value, in descending numerical order
    final_format = (
        merged_dfs.groupby(
            merged_dfs.eraqc_flag_values.apply(type) != str, group_keys=True
        )
        .apply(lambda g: g.sort_values("eraqc_flag_values"))
        .reset_index(drop=True)
    )

    # move string flag value entries to the bottom
    final_format = final_format.loc[
        pd.to_numeric(final_format["eraqc_flag_values"], errors="coerce")
        .sort_values()
        .index
    ]

    # reset index after sorting
    final_format = final_format.reset_index(drop=True)

    # convert all counts to integers
    final_format = final_format.applymap(
        lambda x: int(x) if not isinstance(x, str) else x
    )
    return final_format

In [196]:
def total_sum_flag_counts(df_1,df_2) -> None:
    """
    Sums all network-level QAQC flag counts for a given timestep (hourly or native) and sends to AWS.
    These counts are used to generate statistics for the QAQC success report.

    Parameters
    ----------
    timestep: str
        if set to 'hourly', merge all hourly QAQC flag count tables
        if set to 'native', merge all native timestep QAQC flag count tables

    Returns
    -------
    None

    """

    ## Merge flag counts
    summed_counts_df = _pairwise_sum(df_1, df_2)

    # format final table
    flag_table = pd.read_csv("era_qaqc_flag_meanings.csv")
    final_table = _format_table(summed_counts_df, flag_table, 'total')

    ## Send final counts file to AWS as CSV
    if len(final_table) == 0:
        return None
    else:
        # csv_s3_filepath = f"s3://wecc-historical-wx/4_merge_wx/total_flag_counts_{timestep}_timestep.csv"
        # final_table.to_csv(csv_s3_filepath, index=False)
        # print(f"Sending final summed counts dataframe for to: {csv_s3_filepath}")
        print('you are ok :)')

        return final_table

In [216]:
total_test = total_sum_flag_counts(df_1, df_2)

you are ok :)


In [217]:
total_test

,eraqc_flag_values,elevation,pr,tas,accum_pr,hurs,rsds,tdps_derived,psl,pvp,svp,tdps,ps_altimeter,sfcWind_dir,sfcWind,hurs_derived,ps,QAQC_function,Flag_meaning
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Suspect observation (i.e. buoy reports wind du...
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Out of range for station official data tempora...
2,3,13503,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from DEM (USGS 3DEP)
3,4,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from station
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation manually infilled to be 0.0 m; occur...
5,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height missing
6,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height not 2 meters
7,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height missing
8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height not 10 meters
9,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_precip_logic_nonegvals,Precipitation value reported below 0 (negative...


In [174]:
summed_counts_df = _pairwise_sum(df_1, df_2)

# format final table
flag_table = pd.read_csv("era_qaqc_flag_meanings.csv")
final_table = _format_table(summed_counts_df, flag_table, 'total')

In [175]:
final_table

,eraqc_flag_values,elevation,pr,tas,accum_pr,hurs,rsds,tdps_derived,psl,pvp,svp,tdps,ps_altimeter,sfcWind_dir,sfcWind,hurs_derived,ps,QAQC_function,Flag_meaning
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Suspect observation (i.e. buoy reports wind du...
3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Out of range for station official data tempora...
4,3,13503,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from DEM (USGS 3DEP)
5,4,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from station
6,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation manually infilled to be 0.0 m; occur...
7,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height missing
8,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height not 2 meters
9,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height missing
10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height not 10 meters
11,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_precip_logic_nonegvals,Precipitation value reported below 0 (negative...


step by step

In [155]:
summed_counts = _pairwise_sum(df_1, df_2)

In [169]:
final_table = _format_table(summed_counts, flag_table, "total")

In [170]:
final_table

,eraqc_flag_values,elevation,pr,tas,accum_pr,hurs,rsds,tdps_derived,psl,pvp,svp,tdps,ps_altimeter,sfcWind_dir,sfcWind,hurs_derived,ps,QAQC_function,Flag_meaning
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Suspect observation (i.e. buoy reports wind du...
3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Out of range for station official data tempora...
4,3,13503,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from DEM (USGS 3DEP)
5,4,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from station
6,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation manually infilled to be 0.0 m; occur...
7,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height missing
8,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height not 2 meters
9,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height missing
10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height not 10 meters
11,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,qaqc_precip_logic_nonegvals,Precipitation value reported below 0 (negative...


In [156]:
level = 'total'

In [160]:
## Format flag meanings df
flag_table = flag_table.rename(columns={"Flag_value": "eraqc_flag_values"})

## Format summed counts df

if level == 'network':
    # remove the ".0" from the flag values
    summed_counts["eraqc_flag_values"] = summed_counts["eraqc_flag_values"].str.replace(
        ".0", "", regex=True
    )

# convert flag value strings to integers
summed_counts["eraqc_flag_values"] = summed_counts["eraqc_flag_values"].apply(
    lambda x: int(x) if x not in ["no_flag", "total_obs_count"] else x
)

## Merge the the counts and flag meanings dataframes
merged_dfs = summed_counts.merge(
    flag_table, on="eraqc_flag_values", how="outer"
).fillna(0)

## Format final dataframe

# order by flag value, in descending numerical order
final_format = (
    merged_dfs.groupby(
        merged_dfs.eraqc_flag_values.apply(type) != str, group_keys=True
    )
    .apply(lambda g: g.sort_values("eraqc_flag_values"))
    .reset_index(drop=True)
)

# move string flag value entries to the bottom
final_format = final_format.loc[
    pd.to_numeric(final_format["eraqc_flag_values"], errors="coerce")
    .sort_values()
    .index
]

# convert all counts to integers
final_format = final_format.applymap(
    lambda x: int(x) if not isinstance(x, str) else x
)